In [ ]:
! pip install insightface
! pip install onnxruntime

In [ ]:
import insightface
import numpy as np
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import cv2
import os
import csv

In [66]:
def distance(embeddings1, embeddings2, distance_type='Cosine'):
    embeddings1=embeddings1.astype(np.float64)
    embeddings2=embeddings2.astype(np.float64)
    if distance_type=='Euclidian':
        # Euclidian distance
        embeddings1 = embeddings1/np.linalg.norm(embeddings1, axis=0, keepdims=True)
        embeddings2 = embeddings2/np.linalg.norm(embeddings2, axis=0, keepdims=True)
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),0)
    elif distance_type=='Cosine':
        # Distance based on cosine similarity
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=0)
        norm = np.linalg.norm(embeddings1, axis=0) * np.linalg.norm(embeddings2, axis=0)
        similarity = dot/norm
        similarity = min(1,similarity)
        dist=1-similarity
    else:
        raise 'Undefined distance metric %d' % distance_metric 
    return dist

In [67]:
fr_model = insightface.app.FaceAnalysis()
ctx_id = 0
fr_model.prepare(ctx_id = ctx_id, det_thresh=0.1)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [161]:
img1 = cv2.imread("/volumes/1/subjects/04832d112.jpg") #SUBJECT IMAGE
img1_mated = cv2.imread("/volumes/1/output/frgc_cluster_10.png") #MATED (CLUSTER IMAGE)
img2 = cv2.imread("/volumes/1/thomas/thomas-comparison.jpg") #NON-MATED

face1 = fr_model.get(img1)
face1_mated = fr_model.get(img1_mated)
face2 = fr_model.get(img2)

embedding1 = face1[0].normed_embedding
embedding1_mated = face1_mated[0].normed_embedding
embedding2 = face2[0].normed_embedding

In [163]:
print(f"Mated Comparison Score: {distance(embedding1, embedding2)}")
print(f"Non-Mated Comparison Score: {distance(embedding1, embedding1_mated)}")

Mated Comparison Score: 0.9208778034582
Non-Mated Comparison Score: 0.9638482064180096


In [171]:
with open('/home/ubuntu/trainingfiles/frgc_results_cluster_id_to_subjects.txt', 'r') as f:
    reader = csv.reader(f, delimiter=':')
    clusters_dict = {}
    for i, row in enumerate(reader):
        if i < 2:  # Skip the first two rows
            continue
        cluster_num = int(row[0].strip())
        user_ids = [user_id.strip()[1:-1] for user_id in row[1].strip()[1:-1].split(',')]
        clusters_dict[cluster_num] = user_ids
        
num_higher = 0 # variable to keep track of how many times non-mated comparison score is higher
total_comparisons = 0 # variable to keep track of total number of comparisons

# Loop through the clusters in the dictionary and process the images
for cluster_num, user_ids in clusters_dict.items():
    print(f"Processing cluster {cluster_num} with user IDs {user_ids}")
    for user_id in user_ids:
        print("Processing user ID: ",user_id)
        # Find all images in the folder that match the user_id 
        img_filenames = sorted([filename for filename in os.listdir('/volumes/1/subjects/') if filename.startswith(user_id) and filename.endswith('.jpg')])
        
        # If there are no matching images, skip to the next user_id
        if not img_filenames:
            print("failed")
            continue
        
        # Loop through all images for this user_id
        for i, img_filename in enumerate(img_filenames):
            # Load the current image and the next image
            img_path = os.path.join('/volumes/1/subjects/', img_filename)
            img = cv2.imread(img_path)
            #print(img_filename)
            
            img_mated_filename = 'frgc_cluster_{}.png'.format(cluster_num)
            img_mated_path = os.path.join('/volumes/1/output/', img_mated_filename) 
            img_mated = cv2.imread(img_mated_path)
            #print(img_filenames[(i + 1) % len(img_filenames)])
            
            img2_filename = 'thomas-comparison.jpg'
            img2_path = os.path.join('/volumes/1/thomas/', img2_filename)
            img2 = cv2.imread(img2_path)
            #print(img2_path)
            
            face = fr_model.get(img)
            face_mated = fr_model.get(img_mated)
            
            # Try to get the face embedding for img2
            try:
                face2 = fr_model.get(img2)
                
                embedding = face[0].normed_embedding
                embedding_mated = face_mated[0].normed_embedding
                embedding2 = face2[0].normed_embedding
                non_mated_score = distance(embedding, embedding2)
                mated_score = distance(embedding, embedding_mated)
                print(f"Non-Mated Comparison Score: {distance(embedding, embedding2)}")
                print(f"Mated Comparison Score: {distance(embedding, embedding_mated)}")
                
                total_comparisons += 1
                if non_mated_score > mated_score:
                    num_higher += 1
                        
            except IndexError:
                print(f"No face detected in {img2_path}")
                break
                
    print(f"Total number of comparisons: {total_comparisons}")
    print(f"Number of times non-mated score is higher: {num_higher}")


Processing cluster 0 with user IDs ['02463', '04397', '04513', '04535', '04601', '04810']
Processing user ID:  02463
Processing user ID:  04397
Processing user ID:  04513
Processing user ID:  04535
Processing user ID:  04601
Processing user ID:  04810
Processing cluster 37 with user IDs ['04200', '04337', '04524', '04633', '04817', '04865', '04879', '04927', '04933']
Processing user ID:  04200
Processing user ID:  04337
Processing user ID:  04524
Processing user ID:  04633
Processing user ID:  04817
failed
Processing user ID:  04865
Processing user ID:  04879
Processing user ID:  04927
Processing user ID:  04933
Processing cluster 21 with user IDs ['04201', '04448', '04507', '04542', '04577', '04613', '04859', '04909', '04914']
Processing user ID:  04201
Processing user ID:  04448
Processing user ID:  04507
Processing user ID:  04542
Processing user ID:  04577
Processing user ID:  04613
Processing user ID:  04859
Processing user ID:  04909
Processing user ID:  04914
Processing cluster 